# Extractive Summarization

## Load necessary modules

In [5]:
from datasets import load_dataset
import pandas as pd
import optuna

from Dataset import Dataset

## Load CNN\_dailymail dataset and initiate Dataset data structure

In [6]:
# Load dataset into a variable
CNN_dataset = load_dataset('cnn_dailymail', '3.0.0')

Reusing dataset cnn_dailymail (/home/davide/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


  0%|          | 0/3 [00:00<?, ?it/s]

## Meena and Gopalani strategy comparison

In [7]:
# Meena & Gopalani environment
MG_scores = {
             'comb1': ['TF_ISF_IDF', 'co_occur', 'sent_length'],
             'comb2': ['co_occur', 'sent_length', 'sent_location'],
             'comb3': ['TF_ISF_IDF', 'co_occur', 'sent_length',
                       'sent_location'],
             'comb4': ['sent_length', 'sent_location', 'named_entities',
                       'pos_keywords', 'proper_noun'],
             'comb5': ['co_occur', 'sent_length', 'sent_location',
                       'named_entities', 'pos_keywords', 'proper_noun'],
             'comb6': ['TF_ISF_IDF', 'co_occur', 'sent_length',
                       'sent_location', 'named_entities', 'pos_keywords',
                       'neg_keywords', 'sent_rank'],
             'comb7': ['TF_ISF_IDF', 'co_occur', 'sent_length',
                       'sent_location', 'named_entities', 'pos_keywords',
                       'neg_keywords']
             }

# Meena & Golapani initial test
MG_test = Dataset(name='MG_test_dataset.json')
MG_num_docs = 100
MG_test.build_dataset(CNN_dataset['train'], MG_num_docs)
MG_results = pd.DataFrame()
for comb, scores in MG_scores.items():
    MG_test.process_dataset(scoreList=scores)
    MG_rouge = MG_test.rouge_computation(show=False)
    MG_results = pd.concat([MG_results, MG_rouge.loc['Mean']], axis=1,
                           ignore_index=True)
print(MG_results.T)

processing dataset: :   0%|                      | 27/287113 [00:04<11:16:25,  7.07it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:213: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
processing dataset: :   0%|                     | 100/287113 [00:17<13:47:09,  5.78it/s]


Dataset built in 17.903135538101196[sec]


computing scores: : 100%|█████████████████████████████| 100/100 [00:02<00:00, 49.90it/s]


Dataset processed in: 2.0063[sec]


computing scores: : 100%|█████████████████████████████| 100/100 [00:01<00:00, 90.28it/s]


...

Dataset processed in: 1.9308[sec]


computing scores: : 100%|█████████████████████████████| 100/100 [00:03<00:00, 29.25it/s]


Dataset processed in: 3.4210[sec]


computing scores: : 100%|█████████████████████████████| 100/100 [00:03<00:00, 30.07it/s]

Dataset processed in: 3.3271[sec]
    Rouge-2  Precision  F1-score
0  0.114578   0.035599  0.053768
1  0.070260   0.048146  0.055320
2  0.117600   0.036917  0.055564
3  0.146901   0.075990  0.097490
4  0.144287   0.074426  0.095598
5  0.120043   0.037519  0.056511
6  0.119123   0.037252  0.056104


## All scores computation

A warning is flagged in processing document 27 due to some of its token being unknown by spacy

In [8]:
results = pd.DataFrame(columns=['Rouge-2', 'Precision', 'F1-score'])
for _doc_num in [6, 100, 1000]:
    CNN_processed.process_dataset(CNN_dataset['train'], doc_th=_doc_num)
    res = CNN_processed.rouge_computation(show=False)
    results.loc[_doc_num] = res.loc['Mean'].T
print(results)

computing scores: : 100%|█████████████████████████████████| 6/6 [00:00<00:00, 20.29it/s]


Dataset processed in: 0.2977[sec]


processing dataset: :   0%|                      | 27/287113 [00:04<10:35:30,  7.53it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:213: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
computing scores: : 100%|█████████████████████████████| 100/100 [00:04<00:00, 22.63it/s]


Dataset processed in: 4.4199[sec]


computing scores: : 100%|███████████████████████████| 1000/1000 [02:17<00:00,  7.29it/s]


Dataset processed in: 137.2069[sec]
       Rouge-2  Precision  F1-score
6     0.109382   0.028958  0.045368
100   0.120221   0.037403  0.056382
1000  0.123249   0.038178  0.057659


## Experiment: Maximum among all the computed Sentence location scores 

In [9]:
# Sum of Location scoring strategies
CNN_processed = Dataset('CNN_dataset')
CNN_processed.process_dataset(CNN_dataset['train'], doc_th=100,
                              all_loc_scores=True)
result = CNN_processed.rouge_computation()

processing dataset: :   0%|                      | 27/287113 [00:04<10:31:13,  7.58it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:213: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
computing scores: : 100%|█████████████████████████████| 100/100 [00:04<00:00, 22.02it/s]


Dataset processed in: 4.5430[sec]
                                           Rouge-2  Precision  F1-score
0001d1afc246a7964130f43ae940af6bc6c57f01  0.120000   0.027650  0.044944
0002095e55fcbd3a2f366d9bf92a95433dc305ef  0.111111   0.017964  0.030928
00027e965c8264c35cc1bc55556db388da82b07f  0.027027   0.007246  0.011429
0002c17436637c4fe1837c935c04de47adb18e9a  0.062500   0.016760  0.026432
0003ad6ef0c37534f80b55b4235108024b407f0b  0.031250   0.010526  0.015748
...                                            ...        ...       ...
004f0f8c694c4b546b29565a8993a555537ff561  0.054054   0.016260  0.025000
004fc12e7cd2505a013d96e816afae3f3ce5015d  0.258065   0.109589  0.153846
00504275ede73591d94a6c1f994fd4856610421c  0.019608   0.007812  0.011173
00512126d65bf2a36801e4ef37f28c86c29deb28  0.085106   0.021053  0.033755
Mean                                      0.122133   0.038602  0.057941

[101 rows x 3 columns]


## Experiment: Location scoring with threshold analysis

In [10]:
# Nobata Location Treshold analysis
CNN_processed = Dataset('CNN_dataset')
CNN_processed.build_dataset(CNN_dataset['train'], doc_th=100)
loc_task = pd.DataFrame(columns=['Rouge-2', 'Precision', 'F1-score'])
for x in range(1, 21):
    CNN_processed.process_dataset(loc_th=x, all_loc_scores=True)
    loc_task.loc[x] = CNN_processed.rouge_computation(show=False).loc['Mean']
print(loc_task)

processing dataset: :   0%|                      | 27/287113 [00:04<11:32:11,  6.91it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:213: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
processing dataset: :   0%|                     | 100/287113 [00:17<13:52:21,  5.75it/s]


Dataset built in 17.966793537139893[sec]


computing scores: : 100%|█████████████████████████████| 100/100 [00:04<00:00, 21.61it/s]


Dataset processed in: 4.6294[sec]


computing scores: : 100%|█████████████████████████████| 100/100 [00:04<00:00, 21.97it/s]


...

Dataset processed in: 4.5457[sec]


computing scores: : 100%|█████████████████████████████| 100/100 [00:04<00:00, 22.11it/s]


Dataset processed in: 4.5251[sec]


computing scores: : 100%|█████████████████████████████| 100/100 [00:04<00:00, 21.74it/s]

Dataset processed in: 4.6003[sec]
     Rouge-2  Precision  F1-score
1   0.118954   0.036971  0.055736
2   0.118954   0.036971  0.055736
3   0.121158   0.038052  0.057187
4   0.121158   0.038059  0.057194
5   0.122133   0.038602  0.057941
6   0.121945   0.038567  0.057874
7   0.121004   0.038204  0.057349
8   0.121637   0.038403  0.057654
9   0.119433   0.037367  0.056262
10  0.119859   0.037490  0.056453
11  0.119918   0.037291  0.056203
12  0.119918   0.037293  0.056205
13  0.119918   0.037247  0.056146
14  0.118761   0.036931  0.055651
15  0.122534   0.038073  0.057399
16  0.121784   0.037834  0.057036
17  0.121496   0.037685  0.056844
18  0.121139   0.037262  0.056377
19  0.121139   0.037328  0.056458
20  0.118412   0.036561  0.055260


## Experiment: Location Scoring with treshold analysis using only Nobata et al. method

In [11]:
# Nobata Location Treshold analysis part 2
CNN_processed = Dataset('CNN_dataset')
CNN_processed.build_dataset(CNN_dataset['train'], doc_th=100)
loc_task = pd.DataFrame(columns=['Rouge-2', 'Precision', 'F1-score'])
for x in range(1, 21):
    CNN_processed.process_dataset(loc_th=x, locFilter=[0, 1, 0, 0, 0])  # Only NB1 used
    loc_task.loc[x] = CNN_processed.rouge_computation(show=False).loc['Mean']
print(loc_task)

processing dataset: :   0%|                      | 27/287113 [00:05<10:52:50,  7.33it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:213: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
processing dataset: :   0%|                     | 100/287113 [00:17<14:05:31,  5.66it/s]


Dataset built in 18.285706520080566[sec]


computing scores: : 100%|█████████████████████████████| 100/100 [00:04<00:00, 21.62it/s]


Dataset processed in: 4.6263[sec]


computing scores: : 100%|█████████████████████████████| 100/100 [00:04<00:00, 22.27it/s]


...

Dataset processed in: 4.5080[sec]


computing scores: : 100%|█████████████████████████████| 100/100 [00:04<00:00, 22.07it/s]


Dataset processed in: 4.5323[sec]


computing scores: : 100%|█████████████████████████████| 100/100 [00:04<00:00, 22.42it/s]

Dataset processed in: 4.4619[sec]
     Rouge-2  Precision  F1-score
1   0.119459   0.037184  0.056042
2   0.120271   0.037432  0.056420
3   0.121158   0.038052  0.057187
4   0.121158   0.038059  0.057194
5   0.122133   0.038602  0.057941
6   0.121945   0.038567  0.057874
7   0.121004   0.038204  0.057349
8   0.121637   0.038330  0.057583
9   0.119433   0.037295  0.056191
10  0.119859   0.037418  0.056382
11  0.119918   0.037218  0.056132
12  0.119918   0.037220  0.056134
13  0.118957   0.036909  0.055659
14  0.118761   0.036859  0.055580
15  0.122534   0.038000  0.057329
16  0.121784   0.037754  0.056957
17  0.121496   0.037606  0.056765
18  0.121139   0.037183  0.056298
19  0.121139   0.037249  0.056380
20  0.118412   0.036482  0.055182


## Experiment: Lemma Usage

In [12]:
results = pd.DataFrame(columns=['Rouge-2', 'Precision', 'F1-score'])
CNN_processed = Dataset('CNN_dataset')

_doc_num = 100
for _lemma in [False, True]:
    CNN_processed.process_dataset(CNN_dataset['train'], doc_th=100,
                                  lemma=_lemma, all_loc_scores=True)
    res = CNN_processed.rouge_computation(show=False)
    results.loc[str(_lemma)] = res.loc['Mean'].T
print(results)

processing dataset: :   0%|                      | 27/287113 [00:04<10:41:29,  7.46it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:213: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
computing scores: : 100%|█████████████████████████████| 100/100 [00:04<00:00, 22.65it/s]


Dataset processed in: 4.4157[sec]


computing scores: : 100%|█████████████████████████████| 100/100 [00:04<00:00, 21.03it/s]

Dataset processed in: 4.7554[sec]
        Rouge-2  Precision  F1-score
False  0.122133   0.038602  0.057941
True   0.156094   0.059743  0.085454


## Define optimization task functions

In [13]:
def CNN_scores_best_representation(trial, CNN_dataset, value, pipeline=None, lemma=False):
    scores = set()
    available_scores = CNN_dataset.get_num_weights(True)
    
    i = trial.suggest_int('Number_of_scores', 1, len(available_scores))
    for x in range(i):
        score = trial.suggest_categorical('Score-%d' % x, available_scores)
        scores.add(score)

    for doc in CNN_dataset.documents.values():
        for sentence in doc.sentences.values():
            sent_score = sentence.scores.__dict__.keys()
            for name in sent_score:
                if name not in scores:
                    sentence.scores.__dict__[name] = 0
    
    results = CNN_dataset.rouge_computation(show=False)
    return results.loc['Mean'][value]


def CNN_scores_weights_task(trial, CNN_dataset, value, pipeline=None, lemma=False):
    scores = [x for x in CNN_dataset.get_num_weights(True)]
    weights = [1 for x in range(len(scores))]

    for score in scores:
        idx = scores.index(score)
        weights[idx] = trial.suggest_float(score, -10.0, 10.0, step=0.5)

    results = CNN_dataset.rouge_computation(weights=weights, show=False)
    return results.loc['Mean'][value]

## Experiment: Best Weights with and Without lemma usage optimization task

Results may change due to stochasticity in the optimizer

In [18]:
# Scores weights optimization task
best_weights_res = pd.DataFrame(data=[], columns=['Rouge-2', 'Precision', 'F1-score'])
weights9 = {'False': {}, 'True': {}}
weightsAll = {'False': {}, 'True': {}}
for _lemma in [False, True]:
    CNN_dataset = load_dataset('cnn_dailymail', '3.0.0')
    CNN_processed = Dataset('Weights_finding_optimisation_task.json')
    
    CNN_processed.process_dataset(CNN_dataset['train'], doc_th=100, all_loc_scores=True, lemma=_lemma)
    
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: CNN_scores_weights_task(trial, CNN_processed, 'Precision', lemma=_lemma),
                   n_trials=500)

    weights = list(study.best_params.values())

    weight_res = CNN_processed.rouge_computation(show=False, weights=weights)
    
    best_weights_res.loc[str(_lemma)] = weight_res.loc['Mean'].T
    
    best_param = study.best_params.copy()
    weightsAll[str(_lemma)] = best_param
    for key, value in best_param.items():
        if value >= 9:
            weights9[str(_lemma)].update({key: value})
    
    CNN_processed.process_dataset(lemma=_lemma, scoreList=list(weights9[str(_lemma)].keys()))
    weight_9_rouge = CNN_processed.rouge_computation(show=False, weights=weights)
    
    best_weights_res.loc[str(_lemma)+'_9'] = weight_9_rouge.loc['Mean'].T
print(best_weights_res)
print(weightsAll)
print(weights9)

Reusing dataset cnn_dailymail (/home/davide/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


  0%|          | 0/3 [00:00<?, ?it/s]

processing dataset: :   0%|                      | 27/287113 [00:05<10:37:02,  7.51it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:213: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
computing scores: : 100%|█████████████████████████████| 100/100 [00:04<00:00, 21.39it/s]
[I 2021-12-28 16:43:01,171] A new study created in memory with name: no-name-90c18839-00e0-4633-a58d-f6a7365eaddc
[I 2021-12-28 16:43:01,204] Trial 0 finished with value: 0.022435290831850817 and parameters: {'TF': -8.0, 'sent_location': -0.5, 'proper_noun': -0.5, 'co_occur': -0.5, 'sent_similarity': 5.0, 'num_val': 7.0, 'TF_ISF_IDF': -6.5, 'sent_rank': -7.0, 'sent_length': -5.0, 'pos_keywords': 8.5, 'neg_keywords': 1.5, 'thematic_features': 5.5, 'named_entities': -5.5}. Best is trial 0 with value: 0.022435290831850817.
[I 2021-12-28 16:43:01,233] Trial 1 finished with value: 0.023425793139608925 and parameters: {'TF': -6.0, 'sent_locatio

Dataset processed in: 4.6762[sec]


[I 2021-12-28 16:43:01,413] Trial 6 finished with value: 0.016714988122354904 and parameters: {'TF': -2.0, 'sent_location': -6.5, 'proper_noun': -7.0, 'co_occur': -7.5, 'sent_similarity': 7.5, 'num_val': 8.0, 'TF_ISF_IDF': -9.0, 'sent_rank': -9.0, 'sent_length': 8.5, 'pos_keywords': -9.5, 'neg_keywords': 9.5, 'thematic_features': -7.5, 'named_entities': 2.5}. Best is trial 3 with value: 0.04006589609729223.
[I 2021-12-28 16:43:01,457] Trial 7 finished with value: 0.03332636172584098 and parameters: {'TF': -7.5, 'sent_location': -6.0, 'proper_noun': -2.5, 'co_occur': 8.0, 'sent_similarity': -4.5, 'num_val': 7.5, 'TF_ISF_IDF': 7.0, 'sent_rank': -1.5, 'sent_length': 4.5, 'pos_keywords': 4.5, 'neg_keywords': -10.0, 'thematic_features': 4.0, 'named_entities': -4.5}. Best is trial 3 with value: 0.04006589609729223.
[I 2021-12-28 16:43:01,494] Trial 8 finished with value: 0.03269187072761805 and parameters: {'TF': 6.0, 'sent_location': -7.5, 'proper_noun': -7.5, 'co_occur': -4.5, 'sent_simila

...

[I 2021-12-28 16:44:23,814] Trial 426 finished with value: 0.19531220516281356 and parameters: {'TF': 1.0, 'sent_location': 0.5, 'proper_noun': 2.0, 'co_occur': -5.0, 'sent_similarity': 2.0, 'num_val': 0.5, 'TF_ISF_IDF': 0.0, 'sent_rank': -4.0, 'sent_length': 0.0, 'pos_keywords': 5.5, 'neg_keywords': 8.5, 'thematic_features': 3.0, 'named_entities': 6.5}. Best is trial 358 with value: 0.22633035986359487.
[I 2021-12-28 16:44:23,888] Trial 427 finished with value: 0.1557851354041078 and parameters: {'TF': 1.0, 'sent_location': 0.5, 'proper_noun': 2.0, 'co_occur': -5.0, 'sent_similarity': 2.5, 'num_val': 0.0, 'TF_ISF_IDF': -0.5, 'sent_rank': -4.0, 'sent_length': 0.5, 'pos_keywords': 5.5, 'neg_keywords': 8.0, 'thematic_features': 3.5, 'named_entities': 6.0}. Best is trial 358 with value: 0.22633035986359487.
[I 2021-12-28 16:44:23,970] Trial 428 finished with value: 0.10853656511941794 and parameters: {'TF': 1.0, 'sent_location': 0.5, 'proper_noun': 2.5, 'co_occur': -5.0, 'sent_similarity'

[I 2021-12-28 16:44:25,354] Trial 446 finished with value: 0.11514602584669437 and parameters: {'TF': 3.0, 'sent_location': 1.0, 'proper_noun': 1.0, 'co_occur': 6.0, 'sent_similarity': 1.0, 'num_val': 6.0, 'TF_ISF_IDF': 0.5, 'sent_rank': -4.0, 'sent_length': 0.5, 'pos_keywords': 4.0, 'neg_keywords': 9.5, 'thematic_features': 1.5, 'named_entities': 8.5}. Best is trial 358 with value: 0.22633035986359487.
[I 2021-12-28 16:44:25,419] Trial 447 finished with value: 0.19861809452725399 and parameters: {'TF': 4.5, 'sent_location': 1.0, 'proper_noun': 0.5, 'co_occur': 6.5, 'sent_similarity': 0.5, 'num_val': 6.5, 'TF_ISF_IDF': 0.0, 'sent_rank': -3.5, 'sent_length': 0.5, 'pos_keywords': -3.0, 'neg_keywords': 9.5, 'thematic_features': 1.0, 'named_entities': 8.0}. Best is trial 358 with value: 0.22633035986359487.
[I 2021-12-28 16:44:25,481] Trial 448 finished with value: 0.05933813299719819 and parameters: {'TF': 2.5, 'sent_location': 1.0, 'proper_noun': 1.0, 'co_occur': 6.5, 'sent_similarity': 

[I 2021-12-28 16:44:26,784] Trial 466 finished with value: 0.10255419934939015 and parameters: {'TF': 3.5, 'sent_location': 0.5, 'proper_noun': 0.0, 'co_occur': 7.5, 'sent_similarity': 1.5, 'num_val': -7.5, 'TF_ISF_IDF': 0.0, 'sent_rank': -4.0, 'sent_length': -8.5, 'pos_keywords': 3.0, 'neg_keywords': 9.5, 'thematic_features': -1.0, 'named_entities': 7.0}. Best is trial 358 with value: 0.22633035986359487.
[I 2021-12-28 16:44:26,857] Trial 467 finished with value: 0.17833307509903318 and parameters: {'TF': 3.0, 'sent_location': 1.0, 'proper_noun': 1.0, 'co_occur': 0.0, 'sent_similarity': 0.0, 'num_val': 6.5, 'TF_ISF_IDF': 0.0, 'sent_rank': -3.5, 'sent_length': 0.5, 'pos_keywords': -2.5, 'neg_keywords': 9.0, 'thematic_features': 1.0, 'named_entities': 8.0}. Best is trial 358 with value: 0.22633035986359487.
[I 2021-12-28 16:44:26,943] Trial 468 finished with value: 0.06400488724553025 and parameters: {'TF': 4.0, 'sent_location': -0.5, 'proper_noun': 0.0, 'co_occur': 1.0, 'sent_similarit

[I 2021-12-28 16:44:28,340] Trial 486 finished with value: 0.1404268319851416 and parameters: {'TF': 2.5, 'sent_location': 0.5, 'proper_noun': 0.5, 'co_occur': -6.5, 'sent_similarity': 1.0, 'num_val': 5.5, 'TF_ISF_IDF': -1.0, 'sent_rank': -3.0, 'sent_length': 0.0, 'pos_keywords': 1.5, 'neg_keywords': 10.0, 'thematic_features': 1.0, 'named_entities': 9.0}. Best is trial 358 with value: 0.22633035986359487.
[I 2021-12-28 16:44:28,428] Trial 487 finished with value: 0.11125451774765308 and parameters: {'TF': 4.0, 'sent_location': 1.0, 'proper_noun': -0.5, 'co_occur': -7.0, 'sent_similarity': 0.0, 'num_val': -5.0, 'TF_ISF_IDF': 0.5, 'sent_rank': -4.5, 'sent_length': 0.5, 'pos_keywords': 3.5, 'neg_keywords': 9.0, 'thematic_features': 0.5, 'named_entities': 7.0}. Best is trial 358 with value: 0.22633035986359487.
[I 2021-12-28 16:44:28,505] Trial 488 finished with value: 0.18612646110664013 and parameters: {'TF': 3.0, 'sent_location': -0.5, 'proper_noun': 1.5, 'co_occur': -6.0, 'sent_similar

Dataset processed in: 1.8622[sec]
          Rouge-2  Precision  F1-score
False    0.195112   0.145205  0.156385
False_9  0.214817   0.143609  0.163496
True     0.257017   0.226330  0.227765
True_9   0.259181   0.217947  0.225384
{'False': {'TF': -3.5, 'sent_location': 1.5, 'proper_noun': 1.5, 'co_occur': -0.5, 'sent_similarity': 0.5, 'num_val': -4.5, 'TF_ISF_IDF': 0.0, 'sent_rank': 1.5, 'sent_length': -1.0, 'pos_keywords': 7.5, 'neg_keywords': 10.0, 'thematic_features': -7.5, 'named_entities': -4.5}, 'True': {'TF': 1.5, 'sent_location': 0.0, 'proper_noun': 0.5, 'co_occur': -6.5, 'sent_similarity': -0.5, 'num_val': 4.0, 'TF_ISF_IDF': 0.0, 'sent_rank': -2.5, 'sent_length': 0.0, 'pos_keywords': 3.5, 'neg_keywords': 9.0, 'thematic_features': -1.0, 'named_entities': 7.0}}
{'False': {'neg_keywords': 10.0}, 'True': {'neg_keywords': 9.0}}


## Experiment: Best Representing subset, optimization task

Takes 5 minutes c.a

In [20]:
CNN_dataset = load_dataset('cnn_dailymail', '3.0.0')
CNN_processed = Dataset('Weights_finding_optimisation_task.json')

subset_rouge = pd.DataFrame(columns=['Rouge-2', 'Precision', 'F1-score'])
subsets = {}
_num_doc = 100

for _lemma in [False, True]:
    CNN_processed.process_dataset(CNN_dataset['train'], doc_th=_num_doc,
                                  lemma=_lemma, suppress_warnings=True)
    for _value in ['Precision', 'Rouge-2', 'F1-score']:
        study = optuna.create_study(direction='maximize')
        study.optimize(lambda trial: CNN_scores_best_representation(trial,
                                                                    CNN_processed,
                                                                    _value,
                                                                    lemma=_lemma),
                       n_trials=500)

        subset = set()
        for key, value in study.best_params.items():
            if key != 'Number_of_scores':
                subset.add(value)
        
        CNN_processed.process_dataset(lemma=_lemma, scoreList=subset)
        index = _value + '_Lemma_{}'.format(_lemma)
        subsets[index] = subset
        subset_rouge.loc[index] = CNN_processed.rouge_computation(show=False).loc['Mean'].T
print(subset_rouge, '\n')

for key, value in subsets.items():
    print(key, '\t', value)

Reusing dataset cnn_dailymail (/home/davide/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


  0%|          | 0/3 [00:00<?, ?it/s]

processing dataset: :   0%|                      | 27/287113 [00:05<14:06:44,  5.65it/s]/home/davide/Desktop/NLU_Final_Project/Dataset.py:213: UserWarning: [W008] Evaluating Span.similarity based on empty vectors.
  sentence_similarity = sentence.similarity(sent2)
computing scores: : 100%|█████████████████████████████| 100/100 [00:05<00:00, 19.18it/s]
[I 2021-12-28 17:15:48,170] A new study created in memory with name: no-name-e4b95888-b2af-45d7-b0ff-35c07fb6ead3
[I 2021-12-28 17:15:48,204] Trial 0 finished with value: 0.03622509341848173 and parameters: {'Number_of_scores': 6, 'Score-0': 'sent_rank', 'Score-1': 'neg_keywords', 'Score-2': 'TF_ISF_IDF', 'Score-3': 'sent_location', 'Score-4': 'TF_ISF_IDF', 'Score-5': 'co_occur'}. Best is trial 0 with value: 0.03622509341848173.
[I 2021-12-28 17:15:48,230] Trial 1 finished with value: 0.05701359796322159 and parameters: {'Number_of_scores': 4, 'Score-0': 'pos_keywords', 'Score-1': 'pos_keywords', 'Score-2': 'sent_length', 'Score-3': 'sent

Dataset processed in: 5.2149[sec]


[I 2021-12-28 17:15:48,388] Trial 7 finished with value: 0.05701359796322159 and parameters: {'Number_of_scores': 6, 'Score-0': 'proper_noun', 'Score-1': 'thematic_features', 'Score-2': 'num_val', 'Score-3': 'thematic_features', 'Score-4': 'sent_rank', 'Score-5': 'neg_keywords'}. Best is trial 1 with value: 0.05701359796322159.
[I 2021-12-28 17:15:48,416] Trial 8 finished with value: 0.05701359796322159 and parameters: {'Number_of_scores': 3, 'Score-0': 'num_val', 'Score-1': 'named_entities', 'Score-2': 'sent_location'}. Best is trial 1 with value: 0.05701359796322159.
[I 2021-12-28 17:15:48,442] Trial 9 finished with value: 0.05701359796322159 and parameters: {'Number_of_scores': 6, 'Score-0': 'neg_keywords', 'Score-1': 'proper_noun', 'Score-2': 'num_val', 'Score-3': 'sent_similarity', 'Score-4': 'named_entities', 'Score-5': 'neg_keywords'}. Best is trial 1 with value: 0.05701359796322159.
[I 2021-12-28 17:15:48,471] Trial 10 finished with value: 0.05701359796322159 and parameters: {'

...

[I 2021-12-28 17:18:13,964] Trial 433 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 4, 'Score-0': 'num_val', 'Score-1': 'proper_noun', 'Score-2': 'co_occur', 'Score-3': 'sent_rank'}. Best is trial 1 with value: 0.10086838478308793.
[I 2021-12-28 17:18:14,002] Trial 434 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 5, 'Score-0': 'num_val', 'Score-1': 'proper_noun', 'Score-2': 'co_occur', 'Score-3': 'TF', 'Score-4': 'num_val'}. Best is trial 1 with value: 0.10086838478308793.
[I 2021-12-28 17:18:14,041] Trial 435 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 5, 'Score-0': 'num_val', 'Score-1': 'proper_noun', 'Score-2': 'named_entities', 'Score-3': 'neg_keywords', 'Score-4': 'thematic_features'}. Best is trial 1 with value: 0.10086838478308793.
[I 2021-12-28 17:18:14,076] Trial 436 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 4, 'Score-0': 'num_val', 'Score-1': 

[I 2021-12-28 17:18:15,021] Trial 461 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 12, 'Score-0': 'sent_length', 'Score-1': 'thematic_features', 'Score-2': 'thematic_features', 'Score-3': 'sent_length', 'Score-4': 'num_val', 'Score-5': 'thematic_features', 'Score-6': 'num_val', 'Score-7': 'proper_noun', 'Score-8': 'thematic_features', 'Score-9': 'neg_keywords', 'Score-10': 'sent_similarity', 'Score-11': 'sent_rank'}. Best is trial 1 with value: 0.10086838478308793.
[I 2021-12-28 17:18:15,066] Trial 462 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 12, 'Score-0': 'sent_length', 'Score-1': 'thematic_features', 'Score-2': 'TF_ISF_IDF', 'Score-3': 'sent_length', 'Score-4': 'num_val', 'Score-5': 'TF_ISF_IDF', 'Score-6': 'neg_keywords', 'Score-7': 'sent_location', 'Score-8': 'TF_ISF_IDF', 'Score-9': 'pos_keywords', 'Score-10': 'co_occur', 'Score-11': 'sent_rank'}. Best is trial 1 with value: 0.10086838478308793.
[I 2021-12-28 1

[I 2021-12-28 17:18:15,810] Trial 479 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 13, 'Score-0': 'TF_ISF_IDF', 'Score-1': 'TF_ISF_IDF', 'Score-2': 'TF_ISF_IDF', 'Score-3': 'sent_location', 'Score-4': 'co_occur', 'Score-5': 'proper_noun', 'Score-6': 'sent_similarity', 'Score-7': 'sent_length', 'Score-8': 'co_occur', 'Score-9': 'proper_noun', 'Score-10': 'named_entities', 'Score-11': 'thematic_features', 'Score-12': 'thematic_features'}. Best is trial 1 with value: 0.10086838478308793.
[I 2021-12-28 17:18:15,857] Trial 480 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 13, 'Score-0': 'TF_ISF_IDF', 'Score-1': 'TF_ISF_IDF', 'Score-2': 'TF_ISF_IDF', 'Score-3': 'thematic_features', 'Score-4': 'TF', 'Score-5': 'proper_noun', 'Score-6': 'neg_keywords', 'Score-7': 'named_entities', 'Score-8': 'sent_rank', 'Score-9': 'proper_noun', 'Score-10': 'named_entities', 'Score-11': 'sent_rank', 'Score-12': 'sent_similarity'}. Best is trial 

[I 2021-12-28 17:18:16,555] Trial 495 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 13, 'Score-0': 'neg_keywords', 'Score-1': 'TF_ISF_IDF', 'Score-2': 'thematic_features', 'Score-3': 'sent_location', 'Score-4': 'TF', 'Score-5': 'sent_similarity', 'Score-6': 'sent_rank', 'Score-7': 'sent_location', 'Score-8': 'proper_noun', 'Score-9': 'named_entities', 'Score-10': 'sent_location', 'Score-11': 'pos_keywords', 'Score-12': 'TF_ISF_IDF'}. Best is trial 1 with value: 0.10086838478308793.
[I 2021-12-28 17:18:16,588] Trial 496 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 1, 'Score-0': 'neg_keywords'}. Best is trial 1 with value: 0.10086838478308793.
[I 2021-12-28 17:18:16,622] Trial 497 finished with value: 0.10086838478308793 and parameters: {'Number_of_scores': 1, 'Score-0': 'TF_ISF_IDF'}. Best is trial 1 with value: 0.10086838478308793.
[I 2021-12-28 17:18:16,656] Trial 498 finished with value: 0.10086838478308793 and paramete

Dataset processed in: 1.4070[sec]
                        Rouge-2  Precision  F1-score
Precision_Lemma_False  0.176239   0.097535  0.123671
Rouge-2_Lemma_False    0.131074   0.084046  0.094764
F1-score_Lemma_False   0.117136   0.036078  0.054551
Precision_Lemma_True   0.259181   0.217947  0.225384
Rouge-2_Lemma_True     0.135411   0.057234  0.079534
F1-score_Lemma_True    0.113798   0.062328  0.079248 

Precision_Lemma_False 	 {'sent_length', 'pos_keywords'}
Rouge-2_Lemma_False 	 {'sent_rank', 'sent_similarity', 'neg_keywords'}
F1-score_Lemma_False 	 {'thematic_features', 'sent_similarity', 'sent_location', 'proper_noun', 'sent_rank', 'TF_ISF_IDF', 'neg_keywords', 'co_occur'}
Precision_Lemma_True 	 {'neg_keywords'}
Rouge-2_Lemma_True 	 {'sent_similarity', 'sent_length', 'sent_location', 'proper_noun', 'TF_ISF_IDF', 'co_occur'}
F1-score_Lemma_True 	 {'thematic_features', 'sent_length', 'proper_noun', 'sent_rank', 'num_val'}
